In [1]:
# =========================
# 0. 라이브러리 불러오기
# =========================
import pandas as pd
import numpy as np
from scipy import stats

# =========================
# 1. CSV 불러오기
#    (경로는 네 환경에 맞게 수정 가능)
# =========================
ebooks    = pd.read_csv("Ebooks.csv")
orders    = pd.read_csv("Order.csv")
order_book = pd.read_csv("Order_Book.csv")
reading   = pd.read_csv("Reading_data.csv")
reviews   = pd.read_csv("Online_Reviews.csv")
consumers = pd.read_csv("Consumer.csv")
coupons   = pd.read_csv("Coupon.csv")

print("Ebooks:", ebooks.shape)
print("Orders:", orders.shape)
print("Order_Book:", order_book.shape)
print("Reading:", reading.shape)
print("Online_Reviews:", reviews.shape)
print("Consumers:", consumers.shape)
print("Coupons:", coupons.shape)

# =========================
# 2. 기본 전처리 & 파생변수
# =========================

# (1) reading 데이터: member-book 단위 완료 여부 만들기
# 같은 member_id, book_id에 여러 줄 있을 수 있으니 최대 완독률 사용
reading_agg = (
    reading
    .groupby(['member_id', 'book_id'], as_index=False)
    .agg(
        max_comp=('Percentage_Comp', 'max'),
        # 읽은 날짜 중 가장 최근 시점 (필요하면 쓸 수 있음)
        last_read_time=('reading_time_date', 'max')
    )
)

# "완독" 기준: 90% 이상이면 완독으로 정의
reading_agg['completed'] = (reading_agg['max_comp'] >= 90).astype(int)

# (2) 주문 데이터: 쿠폰 사용 여부 플래그
orders['coupon_used'] = orders['coupon_id'].notna().astype(int)

# (3) 주문 + 도서 매핑: order_id – book_id – member_id – coupon_used
order_detail = (
    order_book
    .merge(orders[['order_id', 'member_id', 'order_date', 'coupon_id', 'coupon_used']],
           on='order_id',
           how='left')
)

# (4) member_id + book_id 기준으로 reading + order 매칭
#     -> 이게 우리가 가장 자주 쓸 "분석 단위" (사람-책)
base = (
    reading_agg
    .merge(order_detail, on=['member_id', 'book_id'], how='left')
)

# (5) ebook 정보(장르, 가격 등) 붙이기
base = base.merge(
    ebooks[['book_id', 'bookgenre', 'booksalesprice']],
    on='book_id',
    how='left'
)

# (6) 리뷰 유무 플래그 (원하면 사용)
review_flag = (
    reviews
    .groupby(['member_id', 'book_id'])
    .size()
    .reset_index(name='num_reviews_member_book')
)

base = base.merge(
    review_flag,
    on=['member_id', 'book_id'],
    how='left'
)

base['wrote_review'] = (base['num_reviews_member_book'].fillna(0) > 0).astype(int)

print("\n[Base 데이터 예시]")
print(base.head())
print("\nBase shape:", base.shape)


Ebooks: (69944, 6)
Orders: (5800, 4)
Order_Book: (7697, 3)
Reading: (6657, 4)
Online_Reviews: (3903, 4)
Consumers: (210664, 3)
Coupons: (40, 4)

[Base 데이터 예시]
                              member_id    book_id  max_comp   last_read_time  \
0  000082D9-E62A-45F3-B885-47668D96660D  170201544     100.0   2/13/2017 1:23   
1  002326C3-A6BA-498D-B4F8-8D1FAACFB4EB  130600464     100.0  2/10/2017 23:09   
2  002326C3-A6BA-498D-B4F8-8D1FAACFB4EB  130600465      12.0  2/11/2017 14:35   
3  002326C3-A6BA-498D-B4F8-8D1FAACFB4EB  141008163      53.0   2/9/2017 23:09   
4  002326C3-A6BA-498D-B4F8-8D1FAACFB4EB  141008164      99.5  2/10/2017 15:56   

   completed  order_id  sales_price        order_date  coupon_id  coupon_used  \
0          1  36533208          3.0   2017-02-11 0:08      103.0            1   
1          1  36523047          0.9  2017-02-10 19:34        NaN            0   
2          0  36531264          0.9  2017-02-10 23:09        NaN            0   
3          0  36485539        

In [2]:
# =========================
# 3. t-test (두 그룹 비교)
# =========================
def run_ttest_ind(df, group_col, value_col, group1_value, group2_value):
    """
    df: 데이터프레임
    group_col: 그룹을 나누는 변수명 (예: 'completed')
    value_col: 비교할 숫자 변수 (예: 'coupon_used')
    group1_value: 그룹1 조건값 (예: 1)
    group2_value: 그룹2 조건값 (예: 0)
    """

    g1 = df.loc[df[group_col] == group1_value, value_col].dropna()
    g2 = df.loc[df[group_col] == group2_value, value_col].dropna()

    print(f"\n[독립 2표본 t-검정] {group_col} = {group1_value} vs {group2_value} 에 대한 {value_col}")
    print(f"  그룹1 n={len(g1)}, 평균={g1.mean():.4f}")
    print(f"  그룹2 n={len(g2)}, 평균={g2.mean():.4f}")

    # 등분산 가정 먼저 확인하고 싶으면 levene test 써도 됨
    t_stat, p_val = stats.ttest_ind(g1, g2, equal_var=False)  # Welch t-test (등분산 가정 안함)
    print(f"  t-statistic = {t_stat:.4f}, p-value = {p_val:.4f}")
    return t_stat, p_val


# =========================
# 4. ANOVA (3그룹 이상 평균 비교)
# =========================
def run_anova(df, group_col, value_col):
    """
    df: 데이터프레임
    group_col: 그룹 변수 (예: 'bookgenre')
    value_col: 숫자 변수 (예: 'max_comp')
    """

    groups = []
    labels = []
    for g, sub in df.groupby(group_col):
        vals = sub[value_col].dropna()
        if len(vals) > 0:
            groups.append(vals)
            labels.append(g)

    print(f"\n[일원분산분석(ANOVA)] {group_col} 그룹별 {value_col} 평균 비교")
    for lab, vals in zip(labels, groups):
        print(f"  그룹 {lab}: n={len(vals)}, mean={vals.mean():.4f}")

    f_stat, p_val = stats.f_oneway(*groups)
    print(f"  F-statistic = {f_stat:.4f}, p-value = {p_val:.4f}")
    return f_stat, p_val


In [3]:
# ====================================
# 0번 연구 질문: 완독 vs 미완독 쿠폰 사용률 차이
# ====================================

# 분석용 df 구성
df_q0 = base[['member_id', 'book_id', 'completed', 'coupon_used']].dropna(subset=['coupon_used']).copy()
df_q0['coupon_used'] = df_q0['coupon_used'].astype(int)

# t-test 실행
run_ttest_ind(
    df=df_q0,
    group_col='completed',
    value_col='coupon_used',
    group1_value=1,   # 완독
    group2_value=0    # 미완독
)


[독립 2표본 t-검정] completed = 1 vs 0 에 대한 coupon_used
  그룹1 n=4834, 평균=0.4820
  그룹2 n=1839, 평균=0.5318
  t-statistic = -3.6411, p-value = 0.0003


(np.float64(-3.6411010439354947), np.float64(0.00027561359914050817))

In [4]:
# ===========================================
# 완독률을 25% 단위로 4그룹 나누기 + ANOVA
# ===========================================

# 분석용 데이터 정리
df_q = base[['member_id', 'book_id', 'max_comp', 'coupon_used']].dropna(subset=['coupon_used', 'max_comp']).copy()
df_q['coupon_used'] = df_q['coupon_used'].astype(int)

# max_comp가 0~100 사이 퍼센트라고 가정하고 4구간으로 자르기
bins = [-0.1, 25, 50, 75, 100.0001]  # 경계 겹치지 않게 살짝 여유
labels = ['G1_0_25', 'G2_25_50', 'G3_50_75', 'G4_75_100']

df_q['comp_group4'] = pd.cut(
    df_q['max_comp'],
    bins=bins,
    labels=labels,
    include_lowest=True
)

print("그룹별 개수:")
print(df_q['comp_group4'].value_counts().sort_index())

print("\n예시 5행:")
print(df_q[['max_comp', 'comp_group4', 'coupon_used']].head())

# ===========================================
# 4그룹의 쿠폰 사용률 평균 차이 ANOVA
# ===========================================
run_anova(
    df=df_q,
    group_col='comp_group4',
    value_col='coupon_used'
)


그룹별 개수:
comp_group4
G1_0_25       990
G2_25_50      281
G3_50_75      336
G4_75_100    5066
Name: count, dtype: int64

예시 5행:
   max_comp comp_group4  coupon_used
0     100.0   G4_75_100            1
1     100.0   G4_75_100            0
2      12.0     G1_0_25            0
3      53.0    G3_50_75            1
4      99.5   G4_75_100            1

[일원분산분석(ANOVA)] comp_group4 그룹별 coupon_used 평균 비교
  그룹 G1_0_25: n=990, mean=0.5222
  그룹 G2_25_50: n=281, mean=0.5374
  그룹 G3_50_75: n=336, mean=0.5417
  그룹 G4_75_100: n=5066, mean=0.4852
  F-statistic = 3.2740, p-value = 0.0202


/var/folders/gx/_f61ngf91hq2y51c1jv2qkkr0000gn/T/ipykernel_44754/2985520626.py:38: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  for g, sub in df.groupby(group_col):


(np.float64(3.2740161329829442), np.float64(0.02020081152501268))

In [5]:
# ===========================================
# 완독률을 10% 단위로 10그룹 나누기 + ANOVA
# ===========================================

# 분석용 데이터 정리
df_q10 = base[['member_id', 'book_id', 'max_comp', 'coupon_used']].dropna(subset=['coupon_used', 'max_comp']).copy()
df_q10['coupon_used'] = df_q10['coupon_used'].astype(int)

# 0~100%를 10% 단위로 등분
bins = [-0.1,10,20,30,40,50,60,70,80,90,100.0001]
labels = [
    'G0_00_10','G1_10_20','G2_20_30','G3_30_40','G4_40_50',
    'G5_50_60','G6_60_70','G7_70_80','G8_80_90','G9_90_100'
]

df_q10['comp_group10'] = pd.cut(
    df_q10['max_comp'],
    bins=bins,
    labels=labels,
    include_lowest=True
)

print("그룹별 개수:")
print(df_q10['comp_group10'].value_counts().sort_index())

# ===========================================
# 10그룹의 쿠폰 사용률 평균 차이 ANOVA
# ===========================================
run_anova(
    df=df_q10,
    group_col='comp_group10',
    value_col='coupon_used'
)


그룹별 개수:
comp_group10
G0_00_10      747
G1_10_20      172
G2_20_30      129
G3_30_40      108
G4_40_50      115
G5_50_60      132
G6_60_70      132
G7_70_80      135
G8_80_90      187
G9_90_100    4816
Name: count, dtype: int64

[일원분산분석(ANOVA)] comp_group10 그룹별 coupon_used 평균 비교
  그룹 G0_00_10: n=747, mean=0.5181
  그룹 G1_10_20: n=172, mean=0.5407
  그룹 G2_20_30: n=129, mean=0.5349
  그룹 G3_30_40: n=108, mean=0.5463
  그룹 G4_40_50: n=115, mean=0.5217
  그룹 G5_50_60: n=132, mean=0.5227
  그룹 G6_60_70: n=132, mean=0.5379
  그룹 G7_70_80: n=135, mean=0.5037
  그룹 G8_80_90: n=187, mean=0.5989
  그룹 G9_90_100: n=4816, mean=0.4817
  F-statistic = 2.0238, p-value = 0.0329


/var/folders/gx/_f61ngf91hq2y51c1jv2qkkr0000gn/T/ipykernel_44754/2985520626.py:38: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  for g, sub in df.groupby(group_col):


(np.float64(2.023772749095934), np.float64(0.03293748718191785))

In [6]:
# ===========================================
# 1번 연구질문: 완독률과 재구매율 관계 분석
# ===========================================

# -----------------------------
# (1) member_id별 구매 책 수 계산
# -----------------------------
member_purchase_counts = (
    order_book
    .merge(orders[['order_id', 'member_id']], on='order_id', how='left')
    .groupby('member_id')
    .size()
    .reset_index(name='num_books_purchased')
)

# 재구매 여부: 2권 이상 = 1, 아니면 0
member_purchase_counts['repurchase'] = (member_purchase_counts['num_books_purchased'] >= 2).astype(int)

# -----------------------------
# (2) 완독 여부(0/1)와 병합
# reading_agg: member_id + book_id 단위라 member_id 단위로 집계 필요
# -----------------------------
member_completion = (
    reading_agg
    .groupby('member_id')
    .agg(
        avg_completion=('max_comp', 'mean'),
        completion_binary=('completed', 'max')   # 1권이라도 완독하면 완독자 그룹
    )
    .reset_index()
)

# -----------------------------
# (3) 병합하여 분석용 DF 생성
# -----------------------------
df_q1 = member_purchase_counts.merge(member_completion, on='member_id', how='left')

# 결측 제거
df_q1 = df_q1.dropna(subset=['completion_binary', 'repurchase'])

# -----------------------------
# (4) t-test 실행 (완독자 vs 미완독자 재구매율 비교)
# -----------------------------
run_ttest_ind(
    df=df_q1,
    group_col='completion_binary',
    value_col='repurchase',
    group1_value=1,   # 완독자
    group2_value=0    # 미완독자
)



[독립 2표본 t-검정] completion_binary = 1 vs 0 에 대한 repurchase
  그룹1 n=2074, 평균=0.6490
  그룹2 n=628, 평균=0.3296
  t-statistic = 14.8534, p-value = 0.0000


(np.float64(14.853365439104975), np.float64(1.9593897082137814e-45))

In [7]:
# ======================================================
# 2번 연구질문(수정): 시간 기반으로 재구매 속도 계산
# ======================================================

# (1) order + coupon 정보 결합
order_df = orders[['order_id', 'member_id', 'order_date', 'coupon_used']].copy()

# (2) order_date를 날짜 + 시간 단위로 변환
order_df['order_date'] = pd.to_datetime(order_df['order_date'], errors='coerce')

# (3) member별로 주문 정렬 후 다음 구매까지 시간 차이 계산
order_df = order_df.sort_values(['member_id', 'order_date'])

order_df['time_to_next_order'] = (
    order_df.groupby('member_id')['order_date'].shift(-1) - order_df['order_date']
)

# timedelta → total hours (또는 total days)
order_df['hours_to_next_order'] = order_df['time_to_next_order'].dt.total_seconds() / 3600
order_df['days_to_next_order'] = order_df['time_to_next_order'].dt.total_seconds() / 86400

# (4) 다음 구매가 실제 존재하는 행만 사용
df_q2 = order_df.dropna(subset=['hours_to_next_order']).copy()

# (5) t-test: 쿠폰 여부 vs 다음 구매까지 걸린 '시간'
run_ttest_ind(
    df=df_q2,
    group_col='coupon_used',
    value_col='hours_to_next_order',
    group1_value=1,
    group2_value=0
)



[독립 2표본 t-검정] coupon_used = 1 vs 0 에 대한 hours_to_next_order
  그룹1 n=1214, 평균=9.4518
  그룹2 n=1731, 평균=5.0931
  t-statistic = 14.0317, p-value = 0.0000


(np.float64(14.031690097820618), np.float64(1.3580282278182798e-42))

In [8]:
# ======================================================
# 가격과 쿠폰 사용 여부의 관계 분석 (t-test)
# ======================================================

# 분석용 DF 준비: base + 가격 정보 포함돼 있음
df_price = base[['member_id', 'book_id', 'booksalesprice', 'coupon_used']].dropna(subset=['booksalesprice', 'coupon_used']).copy()
df_price['coupon_used'] = df_price['coupon_used'].astype(int)

# t-test: 쿠폰 사용 그룹 vs 미사용 그룹의 평균 가격 차이
run_ttest_ind(
    df=df_price,
    group_col='coupon_used',
    value_col='booksalesprice',
    group1_value=1,   # 쿠폰 사용
    group2_value=0    # 쿠폰 미사용
)



[독립 2표본 t-검정] coupon_used = 1 vs 0 에 대한 booksalesprice
  그룹1 n=3308, 평균=3.7781
  그룹2 n=3365, 평균=3.3012
  t-statistic = 6.2382, p-value = 0.0000


(np.float64(6.238178388309229), np.float64(4.699146332778471e-10))

In [9]:
# ===========================================
# 가격대(booksalesprice)를 4분위로 나눈 후 쿠폰 사용률 비교
# ===========================================

df_price = base[['booksalesprice', 'coupon_used']].dropna().copy()
df_price['coupon_used'] = df_price['coupon_used'].astype(int)

# 가격을 4분위수로 나누기
df_price['price_group4'], bins = pd.qcut(
    df_price['booksalesprice'],
    q=4,
    labels=['P1_Lowest', 'P2_MidLow', 'P3_MidHigh', 'P4_Highest'],
    retbins=True,
    duplicates='drop'
)

print("가격대 경계값(bins):", bins)
print("\n그룹별 개수:")
print(df_price['price_group4'].value_counts().sort_index())

# ANOVA 실행 (가격대별 쿠폰 사용률 평균 차이)
run_anova(
    df=df_price,
    group_col='price_group4',
    value_col='coupon_used'
)


가격대 경계값(bins): [ 0.   2.5  3.2  3.5 67.5]

그룹별 개수:
price_group4
P1_Lowest     1849
P2_MidLow     2245
P3_MidHigh    1231
P4_Highest    1348
Name: count, dtype: int64

[일원분산분석(ANOVA)] price_group4 그룹별 coupon_used 평균 비교
  그룹 P1_Lowest: n=1849, mean=0.3975
  그룹 P2_MidLow: n=2245, mean=0.4757
  그룹 P3_MidHigh: n=1231, mean=0.5768
  그룹 P4_Highest: n=1348, mean=0.5898
  F-statistic = 52.8515, p-value = 0.0000


/var/folders/gx/_f61ngf91hq2y51c1jv2qkkr0000gn/T/ipykernel_44754/2985520626.py:38: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  for g, sub in df.groupby(group_col):


(np.float64(52.85151897932463), np.float64(9.395312793544527e-34))

In [10]:
# ======================================================
# 2-way ANOVA: 완독률 4그룹 × 쿠폰 사용 여부
# 종속변수: booksalesprice (또는 coupon_used도 가능)
# ======================================================

import statsmodels.api as sm
from statsmodels.formula.api import ols

# (1) 완독률 4그룹 만들기 (이미 있다면 생략 가능)
df_anova = base[['max_comp', 'coupon_used', 'booksalesprice']].dropna().copy()
df_anova['coupon_used'] = df_anova['coupon_used'].astype(int)

# 완독률을 25% 단위로 자르기
bins = [-0.1, 25, 50, 75, 100.0001]
labels = ['C1_0_25','C2_25_50','C3_50_75','C4_75_100']

df_anova['comp_group4'] = pd.cut(
    df_anova['max_comp'],
    bins=bins,
    labels=labels,
    include_lowest=True
)

# (2) 2-way ANOVA 모델
model = ols("booksalesprice ~ C(comp_group4) * C(coupon_used)", data=df_anova).fit()

anova_table = sm.stats.anova_lm(model, typ=2)
print(anova_table)


                                     sum_sq      df          F        PR(>F)
C(comp_group4)                  2657.583167     3.0  94.841957  4.167528e-60
C(coupon_used)                   324.677091     1.0  34.760542  3.910843e-09
C(comp_group4):C(coupon_used)     57.283148     3.0   2.044281  1.054317e-01
Residual                       62253.712589  6665.0        NaN           NaN


In [11]:
# ======================================================
# 2-way ANOVA용 그룹별 평균 테이블 생성
# ======================================================

# 분석용 DF (df_anova) 준비되었다고 가정
group_mean_table = (
    df_anova
    .groupby(['comp_group4', 'coupon_used'])['booksalesprice']
    .mean()
    .reset_index()
    .pivot(index='comp_group4', columns='coupon_used', values='booksalesprice')
)

print("=== 각 그룹별 평균 booksalesprice (엑셀 피벗 형태) ===")
print(group_mean_table)


=== 각 그룹별 평균 booksalesprice (엑셀 피벗 형태) ===
coupon_used         0         1
comp_group4                    
C1_0_25      4.650951  5.343443
C2_25_50     4.200769  3.871854
C3_50_75     3.214935  3.698901
C4_75_100    3.016664  3.448902


/var/folders/gx/_f61ngf91hq2y51c1jv2qkkr0000gn/T/ipykernel_44754/2260018601.py:8: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  .groupby(['comp_group4', 'coupon_used'])['booksalesprice']


In [12]:
# ===========================================
# Customer Tier 만들기
# ===========================================

# member별 구매 횟수 계산
member_purchase_counts = (
    order_book
    .merge(orders[['order_id', 'member_id']], on='order_id', how='left')
    .groupby('member_id')
    .size()
    .reset_index(name='num_books_purchased')
)

# 구매 횟수 기준으로 3티어 구분
def categorize_tier(x):
    if x == 1:
        return 'T1_first_time'
    elif x <= 3:
        return 'T2_mid_user'
    else:
        return 'T3_heavy_user'

member_purchase_counts['customer_tier'] = member_purchase_counts['num_books_purchased'].apply(categorize_tier)


In [13]:
# ======================================================
# 멤버별 완독 비율 + 구매횟수 티어 + 2-way ANOVA
# ======================================================

import statsmodels.api as sm
from statsmodels.formula.api import ols

# 1) 멤버별 완독 비율 (completion_rate)
comp_rate_df = (
    reading_agg.groupby('member_id')
    .agg(
        num_completed=('completed', 'sum'),
        num_books_read=('completed', 'count')
    )
    .reset_index()
)
comp_rate_df['completion_rate'] = comp_rate_df['num_completed'] / comp_rate_df['num_books_read']

# 혹시 이상치/결측 방지
comp_rate_df = comp_rate_df.dropna(subset=['completion_rate']).copy()
comp_rate_df['completion_rate'] = comp_rate_df['completion_rate'].clip(0, 1)

# ✅ qcut 대신: 0~1을 4등분하는 고정 구간으로 cut
bins = [0.0, 0.25, 0.5, 0.75, 1.0000001]
labels = ['CR1_0_25','CR2_25_50','CR3_50_75','CR4_75_100']

comp_rate_df['comp_rate_group4'] = pd.cut(
    comp_rate_df['completion_rate'],
    bins=bins,
    labels=labels,
    include_lowest=True,
    right=True
)

print("완독 비율 경계값:", bins)
print(comp_rate_df['comp_rate_group4'].value_counts().sort_index())


# 2) 멤버별 구매 횟수 → 3티어
member_purchase_counts = (
    order_book
    .merge(orders[['order_id', 'member_id']], on='order_id', how='left')
    .groupby('member_id')
    .size()
    .reset_index(name='num_books_purchased')
)

def categorize_tier(x):
    if x == 1:
        return 'T1_first_time'
    elif x <= 3:
        return 'T2_mid_user'
    else:
        return 'T3_heavy_user'

member_purchase_counts['customer_tier'] = member_purchase_counts['num_books_purchased'].apply(categorize_tier)

# 3) 멤버별 쿠폰 사용률
coupon_rate_df = (
    base[['member_id', 'coupon_used']]
    .dropna()
    .groupby('member_id')
    .agg(coupon_used_mean=('coupon_used','mean'))
    .reset_index()
)

# 4) 분석용 DF 병합
df_q = (
    comp_rate_df
    .merge(member_purchase_counts[['member_id','customer_tier']], on='member_id', how='left')
    .merge(coupon_rate_df, on='member_id', how='left')
    .dropna(subset=['comp_rate_group4','customer_tier','coupon_used_mean'])
)

print("샘플 행 확인:")
print(df_q.head())

# 5) 2-way ANOVA: 완독비율그룹 × 티어 → 쿠폰 사용률
model = ols("coupon_used_mean ~ C(comp_rate_group4) * C(customer_tier)", data=df_q).fit()
anova_table = sm.stats.anova_lm(model, typ=2)
print("\n=== 2-way ANOVA 결과 ===")
print(anova_table)

# 6) 엑셀 피벗처럼 그룹별 평균 쿠폰 사용률
pivot_means = (
    df_q.groupby(['comp_rate_group4','customer_tier'])['coupon_used_mean']
    .mean()
    .reset_index()
    .pivot(index='comp_rate_group4', columns='customer_tier', values='coupon_used_mean')
)

print("\n=== 완독비율그룹 × 고객티어별 평균 쿠폰 사용률 ===")
print(pivot_means)


완독 비율 경계값: [0.0, 0.25, 0.5, 0.75, 1.0000001]
comp_rate_group4
CR1_0_25       656
CR2_25_50      285
CR3_50_75      192
CR4_75_100    1569
Name: count, dtype: int64
샘플 행 확인:
                              member_id  num_completed  num_books_read  \
0  000082D9-E62A-45F3-B885-47668D96660D              1               1   
1  002326C3-A6BA-498D-B4F8-8D1FAACFB4EB              2               4   
2  0033E5E1-A950-4253-B154-2A058A49B638              0               1   
3  005683BD-5677-4A5D-9D9F-EC0F75DFBC9A              0               1   
4  005C0863-5D8B-41D7-B511-9F9FF32AB287              1               1   

   completion_rate comp_rate_group4  customer_tier  coupon_used_mean  
0              1.0       CR4_75_100  T3_heavy_user               1.0  
1              0.5        CR2_25_50  T3_heavy_user               0.5  
2              0.0         CR1_0_25  T1_first_time               1.0  
3              0.0         CR1_0_25  T1_first_time               1.0  
4              1.0       CR

/opt/anaconda3/lib/python3.13/site-packages/statsmodels/base/model.py:1894: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 6, but rank is 1
  warnings.warn('covariance of constraints does not have full '
/var/folders/gx/_f61ngf91hq2y51c1jv2qkkr0000gn/T/ipykernel_44754/3557662822.py:86: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df_q.groupby(['comp_rate_group4','customer_tier'])['coupon_used_mean']


In [14]:
# ======================================================
# 고객 티어 + 쿠폰 사용률만 이용한 간단 비교
# ======================================================

# (티어 이미 만드는 코드가 있다고 가정)
tier_df = (
    member_purchase_counts[['member_id','customer_tier']]
    .merge(coupon_rate_df, on='member_id', how='left')
    .dropna(subset=['coupon_used_mean'])
)
run_anova(
    df=tier_df,
    group_col='customer_tier',
    value_col='coupon_used_mean'
)


[일원분산분석(ANOVA)] customer_tier 그룹별 coupon_used_mean 평균 비교
  그룹 T1_first_time: n=1149, mean=0.6771
  그룹 T2_mid_user: n=957, mean=0.6285
  그룹 T3_heavy_user: n=596, mean=0.3841
  F-statistic = 102.3657, p-value = 0.0000


(np.float64(102.36573786872319), np.float64(1.4078630103211082e-43))

In [15]:
# ===========================================
# 티어별 평균 가격 차이 ANOVA
# ===========================================

# 1) 멤버별 평균 구매 가격 계산
price_df = (
    base[['member_id','booksalesprice']]
    .dropna()
    .groupby('member_id')
    .agg(avg_price=('booksalesprice','mean'))
    .reset_index()
)

# 2) 고객 티어 정보 결합
tier_price_df = (
    price_df
    .merge(member_purchase_counts[['member_id','customer_tier']], on='member_id', how='left')
    .dropna(subset=['customer_tier'])
)

# 3) ANOVA
run_anova(
    df=tier_price_df,
    group_col='customer_tier',
    value_col='avg_price'
)

# 4) 엑셀식 그룹별 평균 가격 출력
print("\n=== 티어별 평균 구매 가격 ===")
print(
    tier_price_df.groupby('customer_tier')['avg_price'].mean()
)



[일원분산분석(ANOVA)] customer_tier 그룹별 avg_price 평균 비교
  그룹 T1_first_time: n=1149, mean=4.9840
  그룹 T2_mid_user: n=957, mean=3.8250
  그룹 T3_heavy_user: n=596, mean=3.0700
  F-statistic = 73.9638, p-value = 0.0000

=== 티어별 평균 구매 가격 ===
customer_tier
T1_first_time    4.983951
T2_mid_user      3.825026
T3_heavy_user    3.070022
Name: avg_price, dtype: float64


In [16]:
# ===========================================
# 1) order_id 단위 쿠폰 사용 여부 집계
# ===========================================

# base에 order_id가 있다면:
order_coupon = (
    base[['order_id', 'coupon_used']]
    .dropna()
    .groupby('order_id')
    .agg(coupon_used_order=('coupon_used', 'max'))  # 해당 주문에서 한 번이라도 쓰면 1
    .reset_index()
)

# ===========================================
# 2) orders와 결합 → member별 주문 시계열
# ===========================================

order_seq = (
    orders[['order_id', 'member_id', 'order_date']]  # order_date 컬럼명 맞게 써줘
    .merge(order_coupon, on='order_id', how='left')
    .fillna({'coupon_used_order': 0})   # 쿠폰 정보 없으면 0으로 간주
)

# 날짜 + order_id 기준으로 정렬
order_seq = order_seq.sort_values(['member_id', 'order_date', 'order_id'])


# ===========================================
# 3) 이전 주문 쿠폰 사용 여부(prev_coupon_used) 생성
# ===========================================

order_seq['prev_coupon_used'] = (
    order_seq
    .groupby('member_id')['coupon_used_order']
    .shift(1)
)

# 첫 주문은 이전 주문이 없으므로 제거
pairs = order_seq.dropna(subset=['prev_coupon_used']).copy()
pairs['prev_coupon_used'] = pairs['prev_coupon_used'].astype(int)

# ===========================================
# 4) 전이확률: 이전 주문 쿠폰 여부 → 이번 주문 쿠폰 여부
# ===========================================

trans_table = pd.crosstab(
    pairs['prev_coupon_used'],
    pairs['coupon_used_order'],
    normalize='index'
)

print("=== 이전 주문 쿠폰 사용 여부에 따른 다음 주문 쿠폰 사용률 ===")
print(trans_table)



=== 이전 주문 쿠폰 사용 여부에 따른 다음 주문 쿠폰 사용률 ===
coupon_used_order       0.0       1.0
prev_coupon_used                     
0                  0.753960  0.246040
1                  0.526166  0.473834


In [17]:
run_ttest_ind(
    df=pairs,
    group_col='prev_coupon_used',
    value_col='coupon_used_order',
    group1_value=1,
    group2_value=0
)



[독립 2표본 t-검정] prev_coupon_used = 1 vs 0 에 대한 coupon_used_order
  그룹1 n=1051, 평균=0.4738
  그룹2 n=1894, 평균=0.2460
  t-statistic = 12.4376, p-value = 0.0000


(np.float64(12.4376025571792), np.float64(3.2958641881805425e-34))

In [18]:
# 개수 기준 전이행렬
trans_count = pd.crosstab(
    pairs['prev_coupon_used'],
    pairs['coupon_used_order']
)

print("=== 이전 주문 쿠폰 여부 → 다음 주문 쿠폰 여부 (개수 기준) ===")
print(trans_count)


=== 이전 주문 쿠폰 여부 → 다음 주문 쿠폰 여부 (개수 기준) ===
coupon_used_order   0.0  1.0
prev_coupon_used            
0                  1428  466
1                   553  498


In [19]:
print("\n=== 이전 주문 쿠폰 사용 여부 분포 (전체) ===")
print(pairs['prev_coupon_used'].value_counts())



=== 이전 주문 쿠폰 사용 여부 분포 (전체) ===
prev_coupon_used
0    1894
1    1051
Name: count, dtype: int64


In [20]:
orders.head()

,order_id,member_id,order_date,coupon_id,coupon_used
0,36482624,71D97E36-CA0B-4C95-8195-65D1291D5793,2017-02-09 16:51,NaN,0
1,36482638,2B23EAA3-74F7-4D46-A21B-B787F33F237A,2017-02-09 16:52,103.0,1
2,36482648,9A35783E-D9D9-4923-A0FE-980533E94CED,2017-02-09 16:52,NaN,0
3,36482656,34B5EF80-3314-495E-A61E-E60DDB7A3D58,2017-02-09 16:53,103.0,1
4,36482660,B4A7FCAE-4A8C-4D3A-B3EA-AD33E34B7487,2017-02-09 16:53,NaN,0


In [26]:
final_df = pd.read_excel('t_test_base_5.xlsx')
final_df.head()

,member_id,book_id,Percentage_Comp,completed_flag,gender,age_group,age,book_price,used_coupon_flag,total_orders,customer_tier,repurchase_flag
0,CF9B371E-4862-4F62-8508-2F830B2C7312,101200217,100,1,0,60+,65,4,1,2,2~3 orders,1
1,CF9B371E-4862-4F62-8508-2F830B2C7312,101200218,100,1,0,60+,65,4,1,2,2~3 orders,1
2,F5AA949E-54D9-4EDC-ADB4-F1E259035CC9,110300344,100,1,1,40s,47,3,1,2,2~3 orders,1
3,C023676E-698A-4726-A779-4FA86A87D888,110400587,100,1,0,30s,39,3,1,1,1 order,0
4,E476E03C-EFEC-4F2F-B562-F4153D51B4C6,110400679,100,1,0,40s,49,3,1,2,2~3 orders,1


In [22]:
# run an anova test grouping comp_10 which is completion rate in 10% increments and dependent variable is used_coupon_flag
import statsmodels.api as sm
from statsmodels.formula.api import ols

model = ols("used_coupon_flag ~ C(comp_10)", data=final_df).fit()
anova_table = sm.stats.anova_lm(model, typ=2)
print(anova_table)


                 sum_sq      df         F   PR(>F)
C(comp_10)     8.664759    10.0  3.478546  0.00014
Residual    1648.487149  6618.0       NaN      NaN


In [23]:
run_anova(
    df=final_df,
    group_col='comp_10',
    value_col='used_coupon_flag'
)



[일원분산분석(ANOVA)] comp_10 그룹별 used_coupon_flag 평균 비교
  그룹 0: n=267, mean=0.5730
  그룹 1: n=400, mean=0.5025
  그룹 2: n=214, mean=0.5467
  그룹 3: n=147, mean=0.5102
  그룹 4: n=134, mean=0.5299
  그룹 5: n=155, mean=0.5032
  그룹 6: n=140, mean=0.5643
  그룹 7: n=156, mean=0.4808
  그룹 8: n=190, mean=0.5789
  그룹 9: n=805, mean=0.5429
  그룹 10: n=4021, mean=0.4708
  F-statistic = 3.4785, p-value = 0.0001


(np.float64(3.478545648169136), np.float64(0.00013962800309625121))

In [28]:
# run ttest between repurchase_flag 0 and 1 for completed_flag
run_ttest_ind(
    df=final_df,
    group_col='completed_flag',
    value_col='repurchase_flag',
    group1_value=1,
    group2_value=0
)


[독립 2표본 t-검정] completed_flag = 1 vs 0 에 대한 repurchase_flag
  그룹1 n=4806, 평균=0.7172
  그룹2 n=1823, 평균=0.6039
  t-statistic = 8.6003, p-value = 0.0000


(np.float64(8.600272332106956), np.float64(1.2490750516301352e-17))

In [31]:
# ===========================================
# 가격대(booksalesprice)를 4분위로 나눈 후 쿠폰 사용률 비교
# ===========================================

price_df = final_df[['book_price', 'used_coupon_flag']].dropna().copy()
price_df['used_coupon_flag'] = price_df['used_coupon_flag'].astype(int)

# 가격을 4분위수로 나누기
price_df['price_group4'], bins = pd.qcut(
    price_df['book_price'],
    q=4,
    labels=['P1_Lowest', 'P2_MidLow', 'P3_MidHigh', 'P4_Highest'],
    retbins=True,
    duplicates='drop'
)

print("가격대 경계값(bins):", bins)
print("\n그룹별 개수:")
print(df_price['price_group4'].value_counts().sort_index())
    
# ANOVA 실행 (가격대별 쿠폰 사용률 평균 차이)
run_anova(
    df=df_price,
    group_col='price_group4',
    value_col='coupon_used'
)


ValueError: Bin labels must be one fewer than the number of bin edges